In [ ]:
# Core libraries for geospatial processing and segmentation
import os
import random
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd

# Segmentation and vectorization libraries
from skimage import segmentation as seg
from rsgislib.vectorutils.createvectors import polygonise_raster_to_vec_lyr

# Geospatial utilities
from rasterio.mask import mask
from shapely.geometry import box
import simplekml

print("All required libraries imported successfully!")

# Cyprus Terrace Detection: Image Segmentation Pipeline

This notebook performs automated image segmentation across the entire Cyprus island using SLIC (Simple Linear Iterative Clustering) algorithm. The workflow includes:

- **Grid-based processing**: Divides Cyprus into polygon grids
- **Adaptive segmentation**: Calculates optimal segment density based on area
- **Multi-format output**: Generates both raster and vector segment representations
- **Automated vectorization**: Converts segments to polygon geometries for analysis

**Key Parameters:**
- Segmentation density: ~47.7 segments per km² (optimized for 10m resolution imagery)
- SLIC compactness: 1 (balanced between spatial and color similarity)
- Gaussian smoothing: σ=2 (noise reduction before segmentation)

**Outputs:**
- Individual polygon segment files in EPSG:32636 (UTM Zone 36N)
- Merged dataset for island-wide analysis

## 1. Data Preparation and Configuration

Verify input data and set up processing parameters for island-wide segmentation.

In [ ]:
# Configuration: Input data paths and parameters
BASE_RASTER_FILE = "all_cyprus3857.tif"  # Full Cyprus imagery in Web Mercator
GRID_POLYGONS_FILE = "grid_Cyprus.gpkg"  # Processing grid polygons
OUTPUT_DIRECTORY = "CyprusMap/polygons_all"  # Output directory for segmented polygons

# Segmentation parameters (optimized for terrace detection)
SEGMENTS_PER_KM2 = 47.658  # Segment density for EPSG:3857 projection
SLIC_COMPACTNESS = 1       # Balance between color/spatial similarity
GAUSSIAN_SIGMA = 2         # Noise reduction parameter
TARGET_CRS = "EPSG:32636"  # UTM Zone 36N for Cyprus

# Verify input raster properties
with rasterio.open(BASE_RASTER_FILE) as src:
    print("=== INPUT RASTER PROPERTIES ===")
    print(f"Coordinate Reference System: {src.crs}")
    print(f"Raster dimensions: {src.width} x {src.height} pixels")
    print(f"Number of bands: {src.count}")
    print(f"Data type: {src.dtypes[0]}")
    print(f"Pixel size: {src.transform[0]:.2f} x {abs(src.transform[4]):.2f} meters")

print(f"\n=== SEGMENTATION CONFIGURATION ===")
print(f"Target segment density: {SEGMENTS_PER_KM2} segments/km²")
print(f"SLIC compactness factor: {SLIC_COMPACTNESS}")
print(f"Gaussian smoothing sigma: {GAUSSIAN_SIGMA}")
print(f"Output CRS: {TARGET_CRS}")

EPSG:3857


In [ ]:
# Load processing grid and verify structure
grid_polygons = gpd.read_file(GRID_POLYGONS_FILE)

print("=== PROCESSING GRID PROPERTIES ===")
print(f"Number of grid polygons: {len(grid_polygons)}")
print(f"Grid CRS: {grid_polygons.crs}")
print(f"Total coverage area: {grid_polygons.geometry.area.sum() / 1e6:.2f} km²")
print(f"Average grid cell area: {grid_polygons.geometry.area.mean() / 1e6:.2f} km²")

# Show column structure
print(f"\nGrid polygon columns: {list(grid_polygons.columns)}")
print("Grid data structure verified successfully!")

## 2. Grid-Based SLIC Segmentation

Process each grid polygon individually using adaptive SLIC segmentation based on area-optimized parameters.

In [ ]:
def process_polygon_segmentation(polygon_data, polygon_index, base_raster_path, output_base_dir):
    """
    Process a single polygon through SLIC segmentation pipeline.
    
    Args:
        polygon_data: Single polygon geometry from GeoDataFrame
        polygon_index (int): Index of the polygon for naming
        base_raster_path (str): Path to the base raster file
        output_base_dir (str): Base output directory
        
    Returns:
        dict: Processing statistics and output paths
    """
    # Create output directory for this polygon
    polygon_output_dir = os.path.join(output_base_dir, f"polygon_{polygon_index}")
    os.makedirs(polygon_output_dir, exist_ok=True)

    # Step 1: Clip raster to polygon bounds
    with rasterio.open(base_raster_path) as src:
        clipped_raster, clipped_transform = mask(src, [polygon_data.geometry], crop=True)
        clipped_profile = src.profile.copy()

    # Update profile for clipped raster
    clipped_profile.update({
        "height": clipped_raster.shape[1],
        "width": clipped_raster.shape[2],
        "transform": clipped_transform
    })

    # Step 2: Calculate optimal number of segments based on area
    pixel_area = abs(clipped_profile['transform'][0] * clipped_profile['transform'][4])
    total_area_m2 = pixel_area * clipped_raster.shape[1] * clipped_raster.shape[2]
    area_km2 = total_area_m2 / 1e6
    
    optimal_segments = max(10, round(SEGMENTS_PER_KM2 * area_km2))  # Minimum 10 segments

    # Step 3: Perform SLIC segmentation
    # Transpose raster from (bands, height, width) to (height, width, bands) for scikit-image
    image_for_segmentation = clipped_raster.transpose(1, 2, 0)
    
    segmented_image = seg.slic(
        image=image_for_segmentation,
        n_segments=optimal_segments,
        compactness=SLIC_COMPACTNESS,
        sigma=GAUSSIAN_SIGMA,
        start_label=1,
        max_num_iter=100,
        convert2lab=True,
        enforce_connectivity=True,
        min_size_factor=0.5,
        max_size_factor=3,
        slic_zero=True
    )

    # Step 4: Save segmented raster (temporary)
    temp_segments_raster = os.path.join(
        polygon_output_dir, 
        f"segments_no{polygon_index}_{optimal_segments}_temp.tif"
    )
    
    segment_profile = clipped_profile.copy()
    segment_profile.update({
        'count': 1, 
        'compress': 'lzw', 
        'dtype': 'uint32', 
        'driver': 'GTiff', 
        'BIGTIFF': 'YES'
    })
    
    with rasterio.open(temp_segments_raster, 'w', **segment_profile) as dst:
        dst.write(segmented_image, 1)

    # Step 5: Vectorize segments to polygons
    temp_vector_path = os.path.join(
        polygon_output_dir, 
        f"segments_no{polygon_index}_temp.gpkg"
    )
    
    polygonise_raster_to_vec_lyr(
        out_vec_file=temp_vector_path,
        out_vec_lyr="segments",
        out_format="GPKG",
        input_img=temp_segments_raster,
        img_band=1,
        mask_img=temp_segments_raster,
        mask_band=1
    )

    # Step 6: Reproject to target CRS and add attributes
    segments_vector = gpd.read_file(temp_vector_path)
    segments_reprojected = segments_vector.to_crs(TARGET_CRS)
    
    # Add processing metadata
    segments_reprojected['grid_id'] = polygon_index
    segments_reprojected['selected'] = 0  # Default selection status
    segments_reprojected['area_m2'] = segments_reprojected.geometry.area
    segments_reprojected['n_segments'] = optimal_segments
    
    # Step 7: Save final output
    final_output_path = os.path.join(
        polygon_output_dir, 
        f"Segments_no{polygon_index}_proj.gpkg"
    )
    segments_reprojected.to_file(final_output_path, driver="GPKG")

    # Step 8: Clean up temporary files
    os.remove(temp_segments_raster)
    os.remove(temp_vector_path)

    return {
        'polygon_id': polygon_index,
        'area_km2': area_km2,
        'n_segments': optimal_segments,
        'n_final_segments': len(segments_reprojected),
        'output_path': final_output_path
    }

# Main processing loop
print("=" * 60)
print("STARTING CYPRUS-WIDE SEGMENTATION PROCESSING")
print("=" * 60)

processing_stats = []
total_polygons = len(grid_polygons)

for polygon_idx, (_, polygon_row) in enumerate(grid_polygons.iterrows(), 1):
    print(f"\nProcessing polygon {polygon_idx}/{total_polygons}...")
    
    try:
        stats = process_polygon_segmentation(
            polygon_data=polygon_row,
            polygon_index=polygon_idx,
            base_raster_path=BASE_RASTER_FILE,
            output_base_dir=OUTPUT_DIRECTORY
        )
        
        processing_stats.append(stats)
        
        print(f"  ✓ Area: {stats['area_km2']:.2f} km²")
        print(f"  ✓ Target segments: {stats['n_segments']}")
        print(f"  ✓ Final segments: {stats['n_final_segments']}")
        
    except Exception as e:
        print(f"  ✗ Error processing polygon {polygon_idx}: {e}")
        continue

# Summary statistics
total_area = sum(s['area_km2'] for s in processing_stats)
total_segments = sum(s['n_final_segments'] for s in processing_stats)
avg_density = total_segments / total_area if total_area > 0 else 0

print(f"\n" + "=" * 60)
print("SEGMENTATION PROCESSING COMPLETED!")
print("=" * 60)
print(f"Processed polygons: {len(processing_stats)}/{total_polygons}")
print(f"Total area processed: {total_area:.2f} km²")
print(f"Total segments created: {total_segments:,}")
print(f"Average segment density: {avg_density:.1f} segments/km²")
print("=" * 60)

## 3. Data Integration and Final Output

Merge all individual polygon segments into a comprehensive island-wide dataset.

In [ ]:
def merge_segmentation_results(input_directory, output_file_path):
    """
    Merge all individual polygon segmentation results into a single comprehensive dataset.
    
    Args:
        input_directory (str): Directory containing individual polygon segment files
        output_file_path (str): Path for the merged output file
        
    Returns:
        dict: Merge statistics and information
    """
    print("=" * 50)
    print("MERGING SEGMENTATION RESULTS")
    print("=" * 50)
    
    # Step 1: Discover all segment files
    segment_files = []
    for root, dirs, filenames in os.walk(input_directory):
        for filename in filenames:
            if filename.endswith('.gpkg') and 'Segments_no' in filename:
                segment_files.append(os.path.join(root, filename))
    
    print(f"Found {len(segment_files)} segment files to merge")
    
    if not segment_files:
        print("No segment files found for merging!")
        return None
    
    # Step 2: Load and merge all segment files
    print("Loading and merging segment files...")
    merged_segments = gpd.GeoDataFrame()
    
    merge_stats = {
        'files_processed': 0,
        'total_segments': 0,
        'files_with_errors': 0,
        'area_coverage_km2': 0
    }
    
    for file_path in segment_files:
        try:
            # Load individual segment file
            segments = gpd.read_file(file_path)
            
            # Add file identifier
            polygon_id = os.path.basename(os.path.dirname(file_path)).replace('polygon_', '')
            segments['source_polygon'] = polygon_id
            
            # Merge with main dataset
            merged_segments = pd.concat([merged_segments, segments], ignore_index=True)
            
            # Update statistics
            merge_stats['files_processed'] += 1
            merge_stats['total_segments'] += len(segments)
            
            if 'area_m2' in segments.columns:
                merge_stats['area_coverage_km2'] += segments['area_m2'].sum() / 1e6
                
        except Exception as e:
            print(f"  ✗ Error processing {file_path}: {e}")
            merge_stats['files_with_errors'] += 1
            continue
    
    # Step 3: Data quality checks and cleanup
    print("Performing data quality checks...")
    
    # Remove any invalid geometries
    initial_count = len(merged_segments)
    merged_segments = merged_segments[merged_segments.geometry.is_valid]
    invalid_removed = initial_count - len(merged_segments)
    
    if invalid_removed > 0:
        print(f"  Removed {invalid_removed} invalid geometries")
    
    # Add unique segment ID
    merged_segments['segment_id'] = range(1, len(merged_segments) + 1)
    
    # Ensure proper CRS
    if merged_segments.crs != TARGET_CRS:
        print(f"  Reprojecting to {TARGET_CRS}")
        merged_segments = merged_segments.to_crs(TARGET_CRS)
    
    # Calculate final area statistics if not present
    if 'area_m2' not in merged_segments.columns:
        merged_segments['area_m2'] = merged_segments.geometry.area
        merge_stats['area_coverage_km2'] = merged_segments['area_m2'].sum() / 1e6
    
    # Step 4: Save merged dataset
    print(f"Saving merged dataset to: {output_file_path}")
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    merged_segments.to_file(output_file_path, driver="GPKG")
    
    # Final statistics
    merge_stats['final_segments'] = len(merged_segments)
    merge_stats['output_file'] = output_file_path
    
    return merge_stats

# Execute merging process
MERGED_OUTPUT_FILE = "Saga_new_data/Cyprus_complete_segments.gpkg"

merge_results = merge_segmentation_results(
    input_directory=OUTPUT_DIRECTORY,
    output_file_path=MERGED_OUTPUT_FILE
)

if merge_results:
    print("\n" + "=" * 50)
    print("MERGING COMPLETED SUCCESSFULLY!")
    print("=" * 50)
    print(f"Files processed: {merge_results['files_processed']}")
    print(f"Files with errors: {merge_results['files_with_errors']}")
    print(f"Total segments: {merge_results['final_segments']:,}")
    print(f"Coverage area: {merge_results['area_coverage_km2']:.2f} km²")
    print(f"Average segment size: {merge_results['area_coverage_km2']*1e6/merge_results['final_segments']:.0f} m²")
    print(f"Output file: {merge_results['output_file']}")
    print("=" * 50)
else:
    print("Merging process failed!")